Movie Recommendation System

Installing necessary libraries

In [111]:
!pip install scikit-learn
!pip install kagglehub
from sklearn.metrics.pairwise import cosine_similarity
import numpy as np
import pandas as pd
import os
import kagglehub
from sklearn.feature_extraction.text import TfidfVectorizer


[notice] A new release of pip is available: 25.0 -> 25.0.1
[notice] To update, run: pip install --upgrade pip

[notice] A new release of pip is available: 25.0 -> 25.0.1
[notice] To update, run: pip install --upgrade pip


The data for this project will be the TMDB 5000 movie dataset from Kaggle. To download, run the following code block.

In [112]:
# Download latest version
path = kagglehub.dataset_download("tmdb/tmdb-movie-metadata")

print("Path to dataset files:", path)

Path to dataset files: /Users/arnavpallem/.cache/kagglehub/datasets/tmdb/tmdb-movie-metadata/versions/2


Basic EDA

In [113]:
data = pd.read_csv(os.path.join(path, "tmdb_5000_movies.csv"))
print(data.shape)
data.head()

(4803, 20)


,budget,genres,homepage,id,keywords,original_language,original_title,overview,popularity,production_companies,production_countries,release_date,revenue,runtime,spoken_languages,status,tagline,title,vote_average,vote_count
0,237000000,"[{""id"": 28, ""name"": ""Action""}, {""id"": 12, ""nam...",http://www.avatarmovie.com/,19995,"[{""id"": 1463, ""name"": ""culture clash""}, {""id"":...",en,Avatar,"In the 22nd century, a paraplegic Marine is di...",150.437577,"[{""name"": ""Ingenious Film Partners"", ""id"": 289...","[{""iso_3166_1"": ""US"", ""name"": ""United States o...",2009-12-10,2787965087,162.0,"[{""iso_639_1"": ""en"", ""name"": ""English""}, {""iso...",Released,Enter the World of Pandora.,Avatar,7.2,11800
1,300000000,"[{""id"": 12, ""name"": ""Adventure""}, {""id"": 14, ""...",http://disney.go.com/disneypictures/pirates/,285,"[{""id"": 270, ""name"": ""ocean""}, {""id"": 726, ""na...",en,Pirates of the Caribbean: At World's End,"Captain Barbossa, long believed to be dead, ha...",139.082615,"[{""name"": ""Walt Disney Pictures"", ""id"": 2}, {""...","[{""iso_3166_1"": ""US"", ""name"": ""United States o...",2007-05-19,961000000,169.0,"[{""iso_639_1"": ""en"", ""name"": ""English""}]",Released,"At the end of the world, the adventure begins.",Pirates of the Caribbean: At World's End,6.9,4500
2,245000000,"[{""id"": 28, ""name"": ""Action""}, {""id"": 12, ""nam...",http://www.sonypictures.com/movies/spectre/,206647,"[{""id"": 470, ""name"": ""spy""}, {""id"": 818, ""name...",en,Spectre,A cryptic message from Bond’s past sends him o...,107.376788,"[{""name"": ""Columbia Pictures"", ""id"": 5}, {""nam...","[{""iso_3166_1"": ""GB"", ""name"": ""United Kingdom""...",2015-10-26,880674609,148.0,"[{""iso_639_1"": ""fr"", ""name"": ""Fran\u00e7ais""},...",Released,A Plan No One Escapes,Spectre,6.3,4466
3,250000000,"[{""id"": 28, ""name"": ""Action""}, {""id"": 80, ""nam...",http://www.thedarkknightrises.com/,49026,"[{""id"": 849, ""name"": ""dc comics""}, {""id"": 853,...",en,The Dark Knight Rises,Following the death of District Attorney Harve...,112.312950,"[{""name"": ""Legendary Pictures"", ""id"": 923}, {""...","[{""iso_3166_1"": ""US"", ""name"": ""United States o...",2012-07-16,1084939099,165.0,"[{""iso_639_1"": ""en"", ""name"": ""English""}]",Released,The Legend Ends,The Dark Knight Rises,7.6,9106
4,260000000,"[{""id"": 28, ""name"": ""Action""}, {""id"": 12, ""nam...",http://movies.disney.com/john-carter,49529,"[{""id"": 818, ""name"": ""based on novel""}, {""id"":...",en,John Carter,"John Carter is a war-weary, former military ca...",43.926995,"[{""name"": ""Walt Disney Pictures"", ""id"": 2}]","[{""iso_3166_1"": ""US"", ""name"": ""United States o...",2012-03-07,284139100,132.0,"[{""iso_639_1"": ""en"", ""name"": ""English""}]",Released,"Lost in our world, found in another.",John Carter,6.1,2124


We have 4803 movies. The meaningful columns for this task are title and overview for providing some text to match with the user query. 

In [109]:
parsed_data = data[['title', 'overview']].copy()

Now we will vectorize the text

In [110]:
vectorizer = TfidfVectorizer(stop_words = 'english')

tfidf_matrix = vectorizer.fit_transform(parsed_data['overview'].values.astype('str'))
tfidf_vectors = tfidf_matrix.toarray().tolist()
parsed_data.loc[:, 'vectors'] = tfidf_vectors


In [102]:
parsed_data.head() #We now can see a column of vectors

,title,overview,vote_average,vectors
0,Avatar,"In the 22nd century, a paraplegic Marine is di...",7.2,"[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, ..."
1,Pirates of the Caribbean: At World's End,"Captain Barbossa, long believed to be dead, ha...",6.9,"[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, ..."
2,Spectre,A cryptic message from Bond’s past sends him o...,6.3,"[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, ..."
3,The Dark Knight Rises,Following the death of District Attorney Harve...,7.6,"[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, ..."
4,John Carter,"John Carter is a war-weary, former military ca...",6.1,"[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, ..."


In [114]:
def cs(entry, df_entry):
    return cosine_similarity(entry, np.array(df_entry).reshape(1, -1))[0][0]

def find_similar_movies(entry, vectorizer, df):
    vector = vectorizer.transform([entry])
    temp_df = pd.DataFrame({
        'title': df['title'],
        'similarity': df['vectors'].apply(lambda x: cs(vector, x))
    })
    
    # Sort the temporary DataFrame and return top 5 titles
    return temp_df.sort_values('similarity', ascending=False).head(5)['title']

In [115]:
def communicate():
    while True:
        entry = input("What movie are you looking for?")
        print(find_similar_movies(entry, vectorizer, parsed_data))

Run this block to repeatedly query the system. Press escape to quit

In [116]:
communicate()

618          Mystery Men
2194      Disaster Movie
1375            Sky High
1451                Zoom
403     Last Action Hero
Name: title, dtype: object


KeyboardInterrupt: 